In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

# Load your dataset
df = pd.read_csv('it_problems.csv')

# Tokenize the problem descriptions
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['problem_description'])
sequences = tokenizer.texts_to_sequences(df['problem_description'])

# Pad the sequences to ensure uniform input length
x_data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=200)

# Convert the solutions to categorical data
y_data = pd.get_dummies(df['solution']).values

# Build the model
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=64, input_length=200),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dense(y_data.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(x_data, y_data, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(x_data, y_data)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Make predictions
new_problems = ["Example problem description"]
new_sequences = tokenizer.texts_to_sequences(new_problems)
new_padded = keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=200)

predictions = model.predict(new_padded)
predicted_solution = np.argmax(predictions, axis=1)

print(predicted_solution)

# Save the model
model.save('it_problem_solver_model.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           640000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 677,509
Trainable params: 677,509
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


KeyboardInterrupt: 

In [4]:
import pandas as pd

# Define the data
data = {
    'problem_description': [
        "How to fix a '404 Not Found' error?",
        "How to recover a lost password on Windows?",
        "How to install Python on Linux?",
        "How to set up a virtual environment in Python?",
        "How to resolve a 'Connection refused' error in a web server?"
    ],
    'solution': [
        "Check if the URL is correct or if the page has been moved.",
        "Use the password reset feature on the login screen.",
        "Use the package manager to install Python, e.g., `sudo apt-get install python3`.",
        "Use `venv` or `virtualenv` to create an isolated environment.",
        "Check if the web server is running and if the firewall settings are correct."
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('it_problems.csv', index=False)


In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the model
model = load_model('it_problem_solver_model.h5')

# Load the dataset for tokenization
df = pd.read_csv('it_problems.csv')
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['problem_description'])

# Function to display the menu
def display_menu():
    print("Welcome to the IT Problem-Solving Game!")
    print("1. Start Game")
    print("2. Exit")
    choice = input("Enter your choice: ")
    return choice

# Function to ask questions to players
def ask_question(question):
    print("\nQuestion: ", question)
    answer1 = input("Player 1, your answer: ")
    answer2 = input("Player 2, your answer: ")
    return answer1, answer2

# Function to evaluate answers with the AI model
def evaluate_with_ai(answer1, answer2, question):
    # Tokenize and pad the answers for the model
    sequences1 = tokenizer.texts_to_sequences([answer1])
    padded_answer1 = pad_sequences(sequences1, maxlen=200)
    sequences2 = tokenizer.texts_to_sequences([answer2])
    padded_answer2 = pad_sequences(sequences2, maxlen=200)

    # Predict the relevance of the answers
    prediction1 = model.predict(padded_answer1)[0]
    prediction2 = model.predict(padded_answer2)[0]

    # Compare the predictions
    score1 = max(prediction1)
    score2 = max(prediction2)

    if score1 < score2:
        return "Player 1 wins!"
    elif score2 < score1:
        return "Player 2 wins!"
    else:
        return "It's a tie!"

# Main function to run the game
def main():
    while True:
        choice = display_menu()
        if choice == '1':
            # Select a random question from the dataset
            question_row = df.sample(n=1).iloc[0]
            question = question_row['problem_description']
            
            # Ask the question and get answers
            answer1, answer2 = ask_question(question)
            
            # Evaluate answers with the AI model and display results
            result = evaluate_with_ai(answer1, answer2, question)
            print(result)
        elif choice == '2':
            print("Thanks for playing!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Welcome to the IT Problem-Solving Game!
1. Start Game
2. Exit

Question:  How to install Python on Linux?
1/1 [==============================] - 0s 36ms/step
Player 1 wins!
Welcome to the IT Problem-Solving Game!
1. Start Game
2. Exit

Question:  How to set up a virtual environment in Python?
1/1 [==============================] - 0s 26ms/step
Player 2 wins!
Welcome to the IT Problem-Solving Game!
1. Start Game
2. Exit

Question:  How to recover a lost password on Windows?
1/1 [==============================] - 0s 30ms/step
Player 2 wins!
Welcome to the IT Problem-Solving Game!
1. Start Game
2. Exit
Thanks for playing!
